## 1. Import Code

In [1]:
# !rm -rf FreewayGame/
!git clone https://oramleo@github.com/DionisiusMayr/FreewayGame.git

Cloning into 'FreewayGame'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 367 (delta 41), reused 68 (delta 29), pack-reused 281
Receiving objects: 100% (367/367), 70.79 MiB | 21.66 MiB/s, done.
Resolving deltas: 100% (166/166), done.


## 2. Import Libraries

In [1]:
import sys
sys.path.append('/content/FreewayGame/marianna/')

%matplotlib inline
from collections import defaultdict
from typing import List
from sklearn.preprocessing import StandardScaler

import time
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots
import src.serializer as serializer

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
modelSavePath = '/content/drive/MyDrive/02-second-semester-unicamp/02-reinforcement-learning/01-project01-rl/models/'
scoresSavePath = '/content/drive/MyDrive/02-second-semester-unicamp/02-reinforcement-learning/01-project01-rl/scores/'

## 3. Auxiliary functions

### 3.1. Reduce state space

In [4]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob

### 3.2. Reward policy

In [5]:
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

### 3.3. Print results

In [6]:
def print_result(i, scores, total_reward, score):
#     if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

## 4. Running algorithms

### 4.1. Hyperparameters

In [7]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

ALPHA = 0.00001
GAMMA = 0.99
AVAILABLE_ACTIONS = 2
N0 = 2.5

reward_policy.REWARD_IF_CROSS = 50
reward_policy.REWARD_IF_COLISION = -1
reward_policy.REWARD_IF_STILL = -0.1

### 4.2. Monte Carlo

In [ ]:
env, initial_state = environment.get_env()

mc_agent = agents.MonteCarloControl(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
def MonteCarloES(agent, reduce_s, reward_p, RAM_mask: List[int], render: bool=False):
    epi = episode.generate_episode(env, reduce_state=reduce_s, reward_policy=reward_p, agent=agent, RAM_mask=RAM_mask, render=render)
    return agent.update_policy(epi)

In [ ]:
%%time
scores = []
total_rewards = []


n_runs = 5

for i in range(n_runs):
    render = i % 201 == 200

    score, total_reward = MonteCarloES(agent=mc_agent,reduce_s=reduce_state, reward_p=reward_policy, RAM_mask=RAM_mask, render=render)

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

Run [   0] - Total reward:  297.50 Mean scores: 10.00 Means Scores[:-10]: 10.00 Score: 10 
Run [   1] - Total reward:  406.80 Mean scores: 11.00 Means Scores[:-10]: 11.00 Score: 12 
Run [   2] - Total reward:  391.80 Mean scores: 11.33 Means Scores[:-10]: 11.33 Score: 12 
Run [   3] - Total reward:  347.60 Mean scores: 11.25 Means Scores[:-10]: 11.25 Score: 11 
Run [   4] - Total reward:  459.00 Mean scores: 11.60 Means Scores[:-10]: 11.60 Score: 13 
CPU times: user 13.1 s, sys: 13.4 ms, total: 13.1 s
Wall time: 13.1 s


### 4.3. Q Learning

In [ ]:
env, initial_state = environment.get_env()
ql_agent = agents.QLearning(gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0)

In [ ]:
%%time
scores = []
total_rewards = []

n_runs = 1
render = False
for i in range(n_runs):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        ql_agent.update_Q(old_state, state, action, reward)

        action = ql_agent.act(state)  # Next action

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)

Run [   0] - Total reward:  392.20 Mean scores: 12.00 Means Scores[:-10]: 12.00 Score: 12 
CPU times: user 2.61 s, sys: 2.66 ms, total: 2.61 s
Wall time: 2.62 s


### 4.4. Q Learning Approximator

In [8]:
feat_type = 'all' #mean
fixed_alpha = True #False
weights_length = len(RAM_mask) #4

env, initial_state = environment.get_env()
ql_agent_app = agents.QLearningLinearApprox(alpha=ALPHA, gamma=GAMMA, available_actions=AVAILABLE_ACTIONS, N0=N0, weights_length=weights_length)
ql_agent_app.trainScaler(env, RAM_mask, feat_type=feat_type)

In [17]:
%%time
scores = []
total_rewards = []

n_runs = 6000
render = False
for i in range(1,n_runs+1):
#     render = i % 200 == 0

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = ql_agent_app.act(state, feat_type=feat_type)
    
    score = 0
    total_reward = 0

    print("Episode ",i)
    while not game_over:
        if render:
            time.sleep(0.025)
            env.render()

        old_state = state
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()
        ql_agent_app.update_W(old_state, state, action, reward, fixed_alpha=fixed_alpha, feat_type=feat_type)
        action = ql_agent_app.act(state, feat_type=feat_type)  # Next action

        # print('------------------------------------')
    scores.append(score)
    total_rewards.append(total_reward)
    print_result(i, scores, total_reward, score)

    if i % 500 == 0:
      exp = serializer.Experiment(ql_agent_app, scores, total_rewards, reduce_state, reward_policy)
      exp.save_experiment(modelSavePath,'model_ql_approximator_'+str(i))
      print("Model saved!")

Episode  1
Run [   1] - Total reward:  465.20 Mean scores: 13.00 Means Scores[:-10]: 13.00 Score: 13 
Episode  2
Run [   2] - Total reward:  473.00 Mean scores: 13.00 Means Scores[:-10]: 13.00 Score: 13 
Model saved!
Score Saved
Episode  3
Run [   3] - Total reward:  523.80 Mean scores: 13.33 Means Scores[:-10]: 13.33 Score: 14 
Episode  4
Run [   4] - Total reward:  407.20 Mean scores: 13.00 Means Scores[:-10]: 13.00 Score: 12 
Model saved!
Score Saved
CPU times: user 18.9 s, sys: 38.7 ms, total: 19 s
Wall time: 19.1 s


In [ ]:
# with open('/content/q_learning_app_fix_2000.dill','wb') as f:
#    dill.dump(obj=ql_agent_app, file=f)

# with open('/content/q_learning_app_fix_2000.dill', "rb") as f:
#   teste = dill.load(file=f)

# teste.W == ql_agent_app.W

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [9]:
# aux_plots.plot_scores(scores)